# 获取电影评论数据集

In [1]:
import pyprind

In [2]:
import pandas as pd

In [4]:
import os

In [10]:
import numpy as np

生成一个50000次迭代的进度条

In [5]:
pbar = pyprind.ProgBar(50000) 

In [6]:
labels = {'pos':1,'neg':0}

In [7]:
df = pd.DataFrame()

对语料库文件夹进行遍历，将测试集test\训练集train\正例样本集pos\负例样本集neg导入dataFrame

In [8]:
for s in ('test','train'):
    for l in ('pos','neg'):
        path = '/Users/fannian/tool_data/aclImdb/%s/%s' % (s,l)
        for file in os.listdir(path):
            with open(os.path.join(path,file), 'r') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()#更新进度条

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:11:34


In [9]:
df.columns = ['review', 'sentiment']

In [11]:
np.random.seed(0)

In [13]:
df = df.reindex(np.random.permutation(df.index))

In [14]:
df.to_csv('/Users/fannian/tool_data/outdata/movie_data.csv',index = False)

In [16]:
df

,review,sentiment
35356,WWE has produced some of the worst pay-per-vie...,1
41124,"This really should deserve a ""O"" rating, or ev...",0
21281,Detective Sergent Vince De Carlo (James Luisi)...,0
4481,"The last good Ernest movie, and the best at th...",1
11103,I must say. This is easily one of my FAVORITE ...,1
2472,"I would not compare it to Le Placard, which IM...",1
35360,"An absolutely brilliant film! Jiri Trnka, the ...",1
47825,"I considered myself to be quite melancholy, es...",0
45431,was sort of enjoying this movie until the issu...,0
11186,One of the best horror/suspense movies I have ...,1
